In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk import tokenize as tknz
from nltk.corpus import stopwords

from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet

from natasha import Doc, MorphVocab, Segmenter, NewsEmbedding, NewsMorphTagger


nltk.download('wordnet')

In [ ]:
class TextProcessor:
    
    def __init__(self,message=None,verbose=False):
        self._message= message
        self._verbose=verbose
        self.tokens=[]

    def set_message(self,msg):
        self._message = msg;
        if self._verbose: print ("  --> The message was set {}".format(self._message))
        return self

    def get_message(self):
        return self._message

    def remove_email_word(self,replace=" "):
        xtr = re.sub(pattern="@[\w]*",repl=replace,string=self._message)
        if self._verbose: print("  --> The @word was removed {} {} ".format(xtr,self._message))
        return self.set_message(msg=xtr)
    
    def remove_punctuation(self,replace=" "):
        xtr = re.sub(pattern=r'[^\w\s]',repl=replace,string=self._message)
        if self._verbose: print("  --> Removing punctuation: {} {}".format(xtr,self._message))
        return self.set_message(msg=xtr)

    def remove_special_sym(self,replace=" "):
        xtr = re.sub(pattern=r'[^a-zA-Z0-9]',repl=replace,string=self._message)
        if self._verbose: print ("  --> Removing special symbols: {} {}".format(xtr,self._message))
        return self.set_message(msg=xtr)
    
    def remove_numbers(self,replace=" "):
        xtr = re.sub(pattern=r'[^a-zA-Z]',repl=replace,string=self._message)
        if self._verbose: print("  --> Removing all numbers: {} {}".format(xtr,self._message))
        return self.set_message(msg=xtr)

    def to_lower (self ):
        xtr = self._message.lower();
        if self._verbose: print ("  --> Making to lower case: {} {}".format(xtr,self._message))
        return self.set_message(msg=xtr)

    def replace_by_dicts (self,dictionary: dict):
        for key in dictionary:
            xtr = self._message.replace(key,dictionary[key])
            #re.sub(pattern=key,repl=dictionary[key],string=self._message)
            if self._verbose:
                print ("  --> Replacing: {}->{} Result: before: {} after: {}".format(key,dictionary[key],self._message,xtr))
            self.set_message(msg=xtr)
        return self

    def escape_single_symbol_words (self):
        xtr=" ".join([word for word in self._message.split() if len(word)>1])
        if self._verbose:
            print ("  --> Escaping single symbol words: {} {}".format(xtr,self._message))
        return self.set_message(msg=xtr)

    def make_tokenization(self,tokenizer):
        self.tokens=tokenizer(self._message)
        if self._verbose:
            print ("  --> custom tokenizer was completed: {}".format(self.tokens))
        return self

    def nltk_word_tokenize(self):
        self.tokens=tknz.word_tokenize(self._message)
        if self._verbose:
            print ("  --> nltk.tokenize.word_tokenize was completed: {}".format(self.tokens))
        return self

    def nltk_word_punc_tokenize(self):
        self.tokens = tknz.wordpunct_tokenize(self._message)
        if self._verbose:
            print ("  --> nltk.tokenize.wordpunct_tokenize was completed: {}".format(self.tokens))
        return self

    def nltk_tok_tok_tokenizer(self):
        self.tokens = tknz.ToktokTokenizer().tokenize(self._message)
        if self._verbose:
            print ("  --> nltk.tokenize.ToktokTokenizer().tokenize was completed: {}".format(self.tokens))
        return self
        
    def nltk_tweet_tokenizer(self):
        self.tokens = tknz.TweetTokenizer().tokenize(self._message)
        if self._verbose:
            print ("  --> nltk.tokenize.TweetTokenizer().tokenize was completed: {}".format(self.tokens))
        return self

    def nltk_with_regexp_tokenizer (self,regexp):
        self.tokens = tknz.RegexpTokenizer(regexp).tokenize(self._message)
        if self._verbose:
            print ("  --> nltk.tokenize.RegexpTokenizer({}).tokenize was completed: {}".format(regexp,self.tokens))
        return self

    def nltk_sentence_tokenizer (self):
        self.tokens = nltk.sent_tokenize(self._message)
        if self._verbose:
            print ("  --> nltk.sent_tokenize() was completed: {}".format(self.tokens))
        return self

    def remove_stopwords_from_tokens (self,lang=None,is_new=True):
        if lang is None:
            sw = set(stopwords.words("english"))
        else :
            sw = set(stopwords.words(lang))
        tks = [token for token in self.tokens if token not in sw]
        if is_new:
            self.tokens_without_stops = tks
        else :
            self.tokens = tks
        return self

    def stemme_tokens (self,stemmer=None,is_new=True):
        if stemmer is None:
            stemmer = PorterStemmer()
        xtr = [stemmer.stem(token) for token in self.tokens]
        if is_new :
            self.stem_tokens = xtr
        else :
            self.tokens = xtr
        return self

    def lematize_tokens (self,lematizer=None,is_new=True):
        if lematizer is None:
            lematizer = WordNetLemmatizer()
        xtr = [lematizer.lemmatize(token) for token in self.tokens]
        if is_new:
            self.lem_tokens = xtr
        else :
            self.tokens = xtr
        return self

In [ ]:
import pandas as pd

file_path_1 = "/home/alex/dev/AiLearning/DataSetStore/twitter_messages_2/negative.csv"
df1 = pd.read_csv(file_path_1,header=None,on_bad_lines='skip',sep=";")

file_path_2 = "/home/alex/dev/AiLearning/DataSetStore/twitter_messages_2/positive.csv"
df2 = pd.read_csv(file_path_2,header=None,on_bad_lines='skip',sep=";")

df = df1._append(df2)
df.head()
df_copy = df.copy()[0:300]

In [ ]:


def make_natasha_tokens(text,
                        morphology_filter_set=None,
                        token_length_limit=None,
                        empty_token="empty",
                        verbose=False):
    segmenter = Segmenter()
    morph_tagger = NewsMorphTagger(NewsEmbedding())
    morph_vocab = MorphVocab()
    
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)

    selected_tokens=[]
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        if verbose:
            print (" >> Input {} ".format(token))
        new_token = None
        if morphology_filter_set is None:
            new_token = token.lemma
        else:
            if token.pos in morphology_filter_set:
                new_token = token.lemma
        if new_token is not None and token_length_limit is not None:
            if len(new_token)<token_length_limit:
                new_token = None
        if new_token is not None:
            selected_tokens.append(new_token)
        if verbose:
            print (" >> Output {} ".format(new_token))
    if len(selected_tokens)==0 :
        selected_tokens.append(empty_token)
    return selected_tokens

make_natasha_tokens(text="Желаю хорошего полёта и удачной посадки,я буду",morphology_filter_set=('NOUN','VERB'), token_length_limit=3,verbose=True)


In [ ]:


def process(txt: str,verbose=False):
    return TextProcessor(verbose=verbose)\
    .set_message(msg=txt)\
    .remove_email_word()\
    .remove_punctuation()\
    .to_lower()\
    .escape_single_symbol_words()\
    .remove_stopwords_from_tokens(lang='russian',is_new=False)\
    .make_tokenization(tokenizer=lambda txt: make_natasha_tokens(text=txt,morphology_filter_set=('NOUN','VERB'),token_length_limit=3))\
    .tokens
    

process(txt="Желаю хорошего полёта и удачной посадки,я буду",verbose=True)

In [ ]:
df_copy['tokens'] = df_copy[3].apply(lambda txt: process(txt=txt,verbose=False))
df_copy.head()

  --> The message was set на работе был полный пиддес :| и так каждое закрытие месяца, я же свихнусь так D:
  --> The @word was removed на работе был полный пиддес :| и так каждое закрытие месяца, я же свихнусь так D: на работе был полный пиддес :| и так каждое закрытие месяца, я же свихнусь так D: 
  --> The message was set на работе был полный пиддес :| и так каждое закрытие месяца, я же свихнусь так D:
  --> Removing punctuation: на работе был полный пиддес    и так каждое закрытие месяца  я же свихнусь так D  на работе был полный пиддес :| и так каждое закрытие месяца, я же свихнусь так D:
  --> The message was set на работе был полный пиддес    и так каждое закрытие месяца  я же свихнусь так D 
  --> Making to lower case: на работе был полный пиддес    и так каждое закрытие месяца  я же свихнусь так d  на работе был полный пиддес    и так каждое закрытие месяца  я же свихнусь так D 
  --> The message was set на работе был полный пиддес    и так каждое закрытие месяца  я же свихнус